In [1]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

from model.YOLOv4 import YOLOv4
import numpy as np
import gc
from tqdm import tqdm
from gpuinfo import GPUInfo

x = 608
y = 608
inp = np.ones((4, x, y, 3), dtype=np.float64)
small, medium, large = YOLOv4(inp, 10)
print(GPUInfo.get_info())
print((x, y), " work and give :", small.shape, medium.shape, large.shape)


Default GPU Device:/device:GPU:0
({}, [71], [5967], [])
(608, 608)  work and give : (4, 76, 76, 30) (4, 38, 38, 30) (4, 19, 19, 30)


In [2]:
from dataset import Dataset

dataset = Dataset(4)
bbox = dataset.yolo_output_to_bbox((small, medium, large))

In [3]:
print(bbox[0][0])

[0.5, 0.006578947368421052, 0.006578947368421052, 0.0, 0.0, array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], dtype=float32)]
